In [20]:
import geopandas as gpd
from shapely.geometry import Point, Polygon, MultiPolygon, LineString
import os
import json
import gzip
import time
import calendar
from datetime import datetime
import numpy as np
import pandas as pd
from datetime import date
import warnings
warnings.filterwarnings('ignore')
SETS = './sets/' # папка с файлами с наборами
LONG = 0.0159 # долгота расстояние 1 км от точки на восток или на запад
LAT = 0.009 # широта расстояние 1 км от точки на север или на юг

In [17]:
def load_dataset(filename): 
    if os.path.exists(SETS + filename + '.gz'):
        with gzip.open(SETS + filename + '.gz', 'rb') as gzip_ref:
            return pd.DataFrame(json.load(gzip_ref))
        
    
def save_dataset(filename):
    data = globals().get(filename)
    if data is not None:
        data = data.to_dict(orient='records')
        json_data = json.dumps(data, ensure_ascii=False, default=default_serializer)
        with gzip.open(SETS + filename + '.gz', 'wb') as gzip_file:
            gzip_file.write(json_data.encode('utf-8'))
        print(f'Сохранено {len(data)} записей в {filename}.gz')
        

# Функция для преобразования данных в геодатафрейм
def create_geodataframe(row, geo_column):
    try:
        geo_data = row[geo_column]

        # Проверяем, является ли geo_data NaN
        if pd.isna(geo_data):
            return gpd.GeoDataFrame(geometry=[None])  # Возвращаем None, если geo_data NaN

        geometry_type = geo_data['type']
        
        if geometry_type == 'Point':
            coordinates = geo_data['coordinates']
            geometry = Point(coordinates)
        elif geometry_type == 'Polygon':
            coordinates = geo_data['coordinates'][0]  # Предполагаем, что координаты переданы в виде (объект)
            geometry = Polygon(coordinates)
        elif geometry_type == 'MultiPolygon':
            geometry = MultiPolygon([Polygon(p[0]) for p in geo_data['coordinates']])
        elif geometry_type == 'LineString':
            coordinates = geo_data['coordinates']
            geometry = LineString(coordinates)
        else:
            # Если тип геометрии не распознан, возвращаем None
            geometry = None
    
    except Exception as e:
        print(f"Ошибка: {e}. Значение: {row[geo_column]}")  # Выводим ошибку и значение, вызвавшее ошибку
        geometry = None  # Возвращаем None в случае ошибки

    return gpd.GeoDataFrame(geometry=[geometry])

# Координаты

В кратце, основная проблема с координатами заключается в разном порядке отображения широты и долготы.

В геоданных с портала data.mos.ru большинство датасетов имеют вид координат:

```
37.723652, 55.814547
```

Но, большинство популярных онлайн карт (goole, OSM) используют "правильный" формат

```
55.814547, 37.723652
```

И такой формат сегодня используют Яндекс карты, хотя относительно недвано они использовали первый "неправильный" вариант. 
Остались следы этих ["железных аргументов"](https://yandex.ru/blog/mapsapi/8089#coord-3) на их сайте. Но, в конечном итоге, они перешли на общепринятые стандарты, а data.mos.ru не перешел, возможно изначально они ориентировались на яндекс, но тот их кинул. data.mos.ru накопив за 11 лет кучу информации не решился изменить старый формат координат.

Более подробной с проблеймой можно знакомится ниже.


## Термины

### Долгота и Широта

#### Долгота

- **Восточная долгота**: измеряется в градусах от 0 до 180 восточно от меридиана Гринвича. Восточная долгота обозначается положительными значениями.
- **Западная долгота**: измеряется в градусах от 0 до 180 западно от меридиана Гринвича. Западная долгота обозначается отрицательными значениями.

#### Широта

- **Северная широта**: измеряется в градусах от 0 до 90 северно от экватора. Северная широта обозначается положительными значениями.
- **Южная широта**: измеряется в градусах от 0 до 90 южно от экватора. Южная широта обозначается отрицательными значениями.

### Параллели и Меридианы

- **Параллели**: это линии на географической сетке, параллельные экватору. Они измеряют широту и обозначают расстояние от экватора в градусах.
- **Меридианы**: это линии, соединяющие северный и южный полюс. Они измеряют долготу и обозначают расстояние от меридиана Гринвича в градусах.

### Меридиан Гринвича

- **Меридиан Гринвича**: это меридиан, проходящий через Гринвич, Лондон. Он служит нулевым меридианом, от которого измеряется долгота. Меридиан Гринвича обозначается с помощью буквы "L". Меридиан Гринвича имеет длину в 1 градус. 

## Форматы представления

Форматы координат LongLat и LatLong относятся к способу представления географических координат, а именно что идет впереди долгота или широта. 

### LongLat (долгота, широта)

- Долгота (Longitude): измеряется в градусах от 0 до 180 восточно и западно от меридиана Гринвича.
- Широта (Latitude): измеряется в градусах от 0 до 90 северно и южно от экватора.

### LatLong (широта, долгота)

- Широта (Latitude): измеряется в градусах от 0 до 90 северно и южно от экватора.
- Долгота (Longitude): измеряется в градусах от 0 до 180 восточно и западно от меридиана Гринвича.

## Системы навигации

- GPS (Global Positioning System) - это глобальная система спутниковой навигации
- ГЛОНАСС (Глобальная Навигационная Спутниковая Система) - российская аналогичная система спутниковой навигации.
- Galileo - европейская система спутниковой навигации.
- BeiDou - китайская система спутниковой навигации.
- IRNSS (Indian Regional Navigation Satellite System) - индийская региональная система спутниковой навигации.


## Расчеты

### Широта

Возьмем "круглую" точку (это где-то в печатниках)

```
55.7 37.7
```

Прибавим одну сотую к широте, на севере в 1.11 км появилась метка

```
55.71 37.7
```

Отнимим сотую, на юге точка в 1.11 км

```
55.69 37.7
```

следовательно 0.01 по широте это 1.11 км. Следовательно по широте **0.009** - это километр

### Долгота

Прибавим одну сотую к долготе, на востоке появилась точка в 629 метрах

```
55.7 37.71
```

Уменьшим сотую, на западе точка в 629 метрах

```
55.7 37.69
```

Следовательно 0.01 по долготе это 629 метров. Следовательно по долготе **0.0159** - это киллометр

In [3]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)

In [4]:
ecology = load_dataset('ecology')
exams = load_dataset('exams')

In [11]:
%%time
ecology['air'] = ecology.apply(lambda row: create_geodataframe(row, 'air') if not pd.isna(row['air']) else None, axis=1)

CPU times: user 2.21 s, sys: 6.76 ms, total: 2.21 s
Wall time: 2.22 s


In [18]:
ecology[ecology['mean'].notna()].sample(5)

,start,end,air,mean,noise,dirt,digging,capital,house_lines,clinics,roads,parks,transports,city_places,live_area,streets,trashes,factories,markets,autostations,gasstations,bad_gasstations,stroi,geo
211,2017-03-01T00:00:00,2017-03-28T00:00:00,None,"{'boundingbox': ['55.7145754', '55.7146577', '37.6051128', '37.605311']}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
192,2017-03-01T00:00:00,2017-03-28T00:00:00,None,"{'boundingbox': ['55.8014344', '55.8041601', '37.5272233', '37.5310365']}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
610,2020-07-01T00:00:00,2020-07-28T00:00:00,None,"{'boundingbox': ['55.7754298', '55.7755298', '37.6837539', '37.6838539']}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
342,2018-04-01T00:00:00,2018-04-28T00:00:00,None,"{'boundingbox': ['55.7197758', '55.7208123', '37.4046344', '37.4069914']}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
668,2024-03-01T00:00:00,2024-03-28T00:00:00,None,"{'boundingbox': ['55.6710348', '55.6734613', '37.7466602', '37.7579786']}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None


In [19]:
exams

ID  \
0      8236   
1      8237   
2      8238   
3      8239   
4      8240   
...     ...   
5537  13773   
5538  13774   
5539  13775   
5540  13776   
5541  13777   

                                                                                                                             EDU_NAME  \
0                             государственное бюджетное образовательное учреждение города москвы школа 390 имени генерала п.и. батова   
1        государственное казенное образовательное учреждение города москвы кадетская школа-интернат 5 преображенский кадетский корпус   
2     государственное казенное образовательное учреждение города москвы специальная (коррекционная) образовательная школа-интернат 31   
3                                           частное учреждение образовательная организация московская интернациональная школа планета   
4                                                         автономная некоммерческая организация средняя образовательная школа колибри   
...                                                                                                                               ...   
5537                                       частное учреждение средняя образовательная школа славянско-англо-американская школа марина   
5538                                                      государственное бюджетное образовательное учреждение города москвы школа 51   
5539                                                    образовательная автономная некоммерческая организация школа права и экономики   
5540                                             государственное бюджетное образовательное учреждение города москвы школа 1883 бутово   
5541                                                    государственное бюджетное образовательное учреждение города москвы школа 1179   

       global_id       YEAR                              AdmArea  \
0      797946035  2016-2017     Восточный административный округ   
1      797949150  2016-2017     Восточный административный округ   
2      797949275  2016-2017     Восточный административный округ   
3      797949425  2016-2017     Восточный административный округ   
4      797949570  2016-2017     Восточный административный округ   
...          ...        ...                                  ...   
5537  2688590137  2023-2024  Юго-Западный административный округ   
5538  2688590138  2023-2024  Юго-Западный административный округ   
5539  2688590139  2023-2024  Юго-Западный административный округ   
5540  2688590140  2023-2024  Юго-Западный административный округ   
5541  2688590141  2023-2024         Южный административный округ   

                       District  PASSES_OVER_220  PASSER_UNDER_160  \
0             район Богородское                5                19   
1             район Богородское               14                47   
2                 район Вешняки                0                 1   
3                 район Вешняки                0                 6   
4     район Восточное Измайлово                1                 5   
...                         ...              ...               ...   
5537        Ломоносовский район               15                17   
5538          район Тёплый Стан                3                18   
5539            район Черёмушки                4                 7   
5540         район Южное Бутово               17                50   
5541   район Чертаново Северное               14                34   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             